## Tópico Complementarar Módulo 13: Seleção de Features

In [1]:
# AED
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# machine learning
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.feature_selection import RFECV, RFE, SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, f1_score

#### Carga dos Dados e RFE - Regressão

Nesta aula, vamos aprender sobre seleção automática de características usando o scikit-learn. Vou explicar três técnicas: Recursive Feature Elimination (RFE), RFE-CV e SelectModule. Vamos importar as bibliotecas necessárias e carregar o dataset de colesterol. Em seguida, vamos treinar o modelo com RFE, que seleciona as características mais importantes. Vou explicar como o RFE funciona, usando um estimador e definindo a quantidade de características desejadas. Vamos analisar as features selecionadas e o ranking das demais. Também vamos avaliar a performance do modelo usando o MinSquaredError. No próximo vídeo, continuaremos explorando outras técnicas de seleção de características.

In [2]:
# carregar os dados - regressão
df_features_selection = pd.read_csv('../../datasets/dataset_colesterol.csv')

# visualizar estrutura de dados
df_features_selection.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  1000 non-null   int64  
 1   Grupo Sanguíneo     996 non-null    object 
 2   Fumante             997 non-null    object 
 3   Nível de Atividade  996 non-null    object 
 4   Idade               997 non-null    float64
 5   Peso                997 non-null    float64
 6   Altura              997 non-null    float64
 7   Colesterol          1000 non-null   float64
dtypes: float64(4), int64(1), object(3)
memory usage: 62.6+ KB


In [3]:
# coletar medidas de variáveis categóricas
moda_grupo_sanguineo = df_features_selection["Grupo Sanguíneo"].mode()
moda_fumante = df_features_selection["Fumante"].mode()
moda_nivel_atividade_fisica = df_features_selection["Nível de Atividade"].mode(
)

# coletar medidade de variáveis numéricas
mediana_idade = df_features_selection["Idade"].median()
mediana_peso = df_features_selection["Peso"].median()
mediana_altura = df_features_selection["Altura"].median()

# imputar valores ausentes
df_features_selection.fillna(
    value={
        "Grupo Sanguíneo": moda_grupo_sanguineo[0],
        "Fumante": moda_fumante[0],
        "Nível de Atividade": moda_nivel_atividade_fisica[0],
        "Idade": mediana_idade,
        "Peso": mediana_peso,
        "Altura": mediana_altura,
    },
    inplace=True,
)

In [4]:
# detectar quantidade valores ausentes (nulos) por coluna
df_features_selection.isna().sum()

Id                    0
Grupo Sanguíneo       0
Fumante               0
Nível de Atividade    0
Idade                 0
Peso                  0
Altura                0
Colesterol            0
dtype: int64

In [5]:
# ajustar dataframe
df_features_selection.drop("Id", axis=1, inplace=True)

# aplicar OneHotEnconding nas variáveis categóricas
df_features_selection = pd.get_dummies(
    df_features_selection, columns=['Grupo Sanguíneo', 'Fumante', 'Nível de Atividade'])

# exibir dataframe atualizado
df_features_selection

,Idade,Peso,Altura,Colesterol,Grupo Sanguíneo_A,Grupo Sanguíneo_AB,Grupo Sanguíneo_B,Grupo Sanguíneo_O,Fumante_Não,Fumante_Sim,Nível de Atividade_Alto,Nível de Atividade_Baixo,Nível de Atividade_Moderado
0,33.0,85.1,186.0,199.63,0,0,1,0,0,1,0,1,0
1,68.0,105.0,184.0,236.98,1,0,0,0,1,0,0,0,1
2,25.0,64.8,180.0,161.79,0,0,0,1,1,0,1,0,0
3,43.0,120.2,167.0,336.24,1,0,0,0,1,0,1,0,0
4,79.0,88.5,175.0,226.23,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,31.0,68.1,166.0,206.81,0,1,0,0,0,1,0,0,1
996,51.0,47.7,170.0,128.03,0,0,0,1,1,0,1,0,0
997,39.0,85.5,176.0,211.14,0,1,0,0,1,0,0,1,0
998,61.0,91.2,161.0,284.53,0,1,0,0,0,1,0,1,0


In [6]:
# separar x e y
X = df_features_selection.drop('Colesterol', axis=1)
y = df_features_selection['Colesterol']

# treinar modelo de regressão linar múltipla
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=51)

In [7]:
# Treina modelo com RFE
# RFE (Recursive Feature Elimination)
# Uso um estimador e defino a quantidade de features (dois hiperparemetros)
# O RFE faz iterações iniciando com todas as features e eliminando cada iteração até atingir a quantidade definida.
# Logo, elimina caracteríticas/features menos importantes
rfe_method = RFE(estimator=LinearRegression(), n_features_to_select=6)
rfe_method.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=6)

In [8]:
# features selecionadas
X_train.columns[(rfe_method.get_support())]

Index(['Peso', 'Altura', 'Grupo Sanguíneo_AB', 'Grupo Sanguíneo_B',
       'Fumante_Sim', 'Nível de Atividade_Alto'],
      dtype='object')

In [9]:
# Ranking de Features

def show_ranking_features_list(method_fs, X_train):
    
    # Obter o Ranking de Features
    ranking = rfe_method.ranking_
    
    features_names = X_train.columns.to_list()
    
    df_ranking = pd.DataFrame({'Feature': features_names, 'Ranking': ranking})
    
    # Ordene o Dataframe pelo ranking 
    df_ranking = df_ranking.sort_values(by='Ranking')
    
    print(df_ranking)
    
# ranking de features do RFE Regressão - exibição da lista com mais e menos inportantes features
show_ranking_features_list(rfe_method, X_train)

                        Feature  Ranking
1                          Peso        1
2                        Altura        1
4            Grupo Sanguíneo_AB        1
5             Grupo Sanguíneo_B        1
8                   Fumante_Sim        1
9       Nível de Atividade_Alto        1
7                   Fumante_Não        2
3             Grupo Sanguíneo_A        3
11  Nível de Atividade_Moderado        4
10     Nível de Atividade_Baixo        5
6             Grupo Sanguíneo_O        6
0                         Idade        7


In [10]:
# função para avaliar performance
def performance_regression(model, X_test, y_test):
    
    # faz predição com o modelo no conjunto de testes
    y_pred = model.predict(X_test)
    
    # avaliar desempenho
    return root_mean_squared_error(y_test, y_pred)

# Performance Regressão com RFE - valor mensurado em erros
performance_regression(rfe_method, X_test, y_test)

8.997293997492445

#### Modelo sem RFE - Regressão

Nesta aula, realizamos o treinamento de um modelo de regressão sem utilizar o método de seleção de características RFE. Utilizamos as funções X-String e Y-String para treinar o modelo e obter o resultado. Em seguida, comparamos a performance desse modelo com a performance de um modelo treinado com RFE, que seleciona apenas duas features. Observamos que o modelo com todas as features obteve um erro menor do que o modelo com RFE. No entanto, ao ajustar a quantidade de features com RFE, conseguimos melhorar a performance do modelo. Isso nos mostra que, em alguns casos, treinar o modelo com menos features pode resultar em uma performance melhor e uma interpretabilidade mais clara.

In [11]:
# treinar modelo de regressão sem RFE
model_no_rfe = LinearRegression()
model_no_rfe.fit(X_train, y_train)

LinearRegression()

In [12]:
# performanc sem RFE
performance_regression(model_no_rfe, X_test, y_test)

9.040245951911174

#### Uso de RFECV - Regressão

In [13]:
# treinamento
rfe_method_cv = RFECV(estimator=LinearRegression(), min_features_to_select=6, cv=5)
rfe_method_cv.fit(X_train, y_train)

RFECV(cv=5, estimator=LinearRegression(), min_features_to_select=6)

In [14]:
# medir performance_
performance_regression(rfe_method_cv, X_test, y_test)

9.047574304437292

In [15]:
# listar features selecionadas
X_train.columns[(rfe_method_cv.get_support())]


Index(['Peso', 'Altura', 'Grupo Sanguíneo_A', 'Grupo Sanguíneo_AB',
       'Grupo Sanguíneo_B', 'Fumante_Não', 'Fumante_Sim',
       'Nível de Atividade_Alto', 'Nível de Atividade_Baixo',
       'Nível de Atividade_Moderado'],
      dtype='object')

In [16]:
# total features selecionadas
rfe_method_cv.n_features_

10

#### Uso de SelectFromModel - Regressão

In [17]:
sfm_method = SelectFromModel(estimator=model_no_rfe, max_features=2)
sfm_method.fit(X_train, y_train)

SelectFromModel(estimator=LinearRegression(), max_features=2)

In [18]:
# Quais features foram selecionadas?
X_train.columns[(sfm_method.get_support())]

Index(['Peso', 'Altura'], dtype='object')

In [19]:
# treinar modelo com as features selecionadas
X_train_adjusted_reg = sfm_method.transform(X_train)
X_test_adjusted_reg = sfm_method.transform(X_test)
model_no_rfe.fit(X_train_adjusted_reg, y_train)

LinearRegression()

In [20]:
# perforance do Modelo SelectFromModel
performance_regression(model_no_rfe, X_test_adjusted_reg, y_test)

9.086733223682526

#### Carga dos Dados e Uso de RFE - Classificação

#### Modelo sem RFE - Classificação

#### Uso de RFECV - Classificação

Neste vídeo, continuamos nosso módulo de seleção de features, treinando um modelo com RFCV. Usamos o código da regressão logística e modificamos algumas coisas. Podemos escolher o número de features selecionadas e a métrica de validação cruzada. Neste caso, escolhemos o F1 weighted. A performance do modelo foi baixa, pois usamos apenas duas features. Podemos ajustar o número de features e ver como isso afeta a performance. O RFCV é útil para selecionar features com base em validação cruzada. No próximo vídeo, aplicaremos o Select From Model na regressão logística.

#### Uso de SelectFromModel - Classificação

Neste vídeo, vamos finalizar o tópico de treinamento de modelos com o Select from Model. Vamos adaptar o modelo de regressão logística para o problema de classificação. Vamos testar o modelo com diferentes quantidades de features e ajustar o threshold. Em seguida, vamos avaliar as features selecionadas e treinar o modelo de regressão logística com essas features. Por fim, vamos comparar a performance desse modelo com o baseline. Também vamos explorar diferentes configurações de features e threshold. No final, concluímos mais um tópico de algoritmos supervisionados.